# Anomaly Detection
# Training stage

The goal here is to train an anomaly detection algorithm in order to find anomalies in our data. As we will see later, once we have trained the model, we can save it and use it on new (streaming) data points. Once new anomalies are found we will get an email notification about them and write them back to Clarify in a new signal.

# Isolation Forest 
To perform anomaly detection, we will use Isolation Forest (or iForest). As mentioned in the [Isolation Forest paper](https://cs.nju.edu.cn/zhouzh/zhouzh.files/publication/icdm08b.pdf?q=isolation-forest) anomalies are here considered to be those which are 'few and different'. These data points can be isolated more easily than 'normal' data points. Therefore anomalies are isolated closer to the root of the tree, also known as Isolation Tree (or iTree) 

<a href="https://www.researchgate.net/figure/Isolation-Forest-learned-iForest-construction-for-toy-dataset_fig1_352017898"><img src="https://www.researchgate.net/publication/352017898/figure/fig1/AS:1029757483372550@1622524724599/Isolation-Forest-learned-iForest-construction-for-toy-dataset.png" alt="Isolation Forest: learned iForest construction for toy dataset"/></a>

[Source](https://www.researchgate.net/figure/Isolation-Forest-learned-iForest-construction-for-toy-dataset_fig1_352017898) Scientific Figure on ResearchGate 


If we create multiple trees we can get the average path lengths from all the data points and from that optain the anomaly score values. The top `m` data points with the corresponding lowest anomaly score values are considered as anomalies.

In order to gain a better understaning of the algorithm's performance, in the following figures we will plot the predict values (which are either 1 or -1) and the average anomaly scores (negative scores represent outliers and positive scores represent normal points).

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from pylab import rcParams
import plotly.graph_objects as go

from sklearn.ensemble import IsolationForest
import pandas as pd
import numpy as np

import orchest
import pickle
import json

# Get data from previous step
orchest_data = orchest.get_inputs()
response = orchest_data["response"]
outliers_fraction = orchest.get_step_param("outliers_fraction")

In [ ]:
dates = response.index
values = sum(response.values.tolist(), [])
df = pd.DataFrame({'date': dates, 'x': values})

X = df[["x"]]

In sklearn the number of trees which will be created are by default 100, and the sub-sampling size is by default 256. 
We will change these values to reduce the run time and the swampling and masking effects on our data points. 

**Swampling** is finding false anomalies. When anomaly points are close to normal points the process of finding these anomaly points is harder and can easiely lead to categorizing normal points as anomalies. 

**Masking** is called the phenomenon where we have many anomalies building a small cluster, therefore ending up to be categorized as normal points. 


These two phenomenons can be tuned by the sub-samplint size parameter (called max_samples in sklearn). Because we don't need to isolate all data points, if we reduce the sub-samlping size, we can obtain more true anomaly points. 
Note that we use sub-sampling without replacement.

<img src="https://raw.githubusercontent.com/clarify/data-science-tutorials/main/media/email_notifications/isolationg.png"/>

[Source Isolation Forest paper](https://cs.nju.edu.cn/zhouzh/zhouzh.files/publication/icdm08b.pdf?q=isolation-forest) 
(a) Isolating a normal point Xi needs more splitting procedures than (b) isolating an anomaly point Xo.


The complexity of the training process can be tuned with the number of trees used (called n_estimators in sklearn). As mentioned in the  [Isolation Forest paper](https://cs.nju.edu.cn/zhouzh/zhouzh.files/publication/icdm08b.pdf?q=isolation-forest) for n_estimators = 100 the path lengths usually converge well. 

<img src="https://raw.githubusercontent.com/clarify/data-science-tutorials/main/media/email_notifications/before.png" width=312 height=312 />  <img src="https://raw.githubusercontent.com/clarify/data-science-tutorials/main/media/email_notifications/after.png" width=300 height=300 />

[Source Isolation Forest paper](https://cs.nju.edu.cn/zhouzh/zhouzh.files/publication/icdm08b.pdf?q=isolation-forest) 
The first plot has 4096 instances, the second plot has 128 instanses


Last but not least, when using Isolation Forest on time series data it usually helps to set the contamination parameter. This parameter defines the threshold on the scores when fitting the data on the model.
The default value is auto, which will set the threshold as determined in the original paper. When using contamination = 'auto' the anomaly points which the algorithm finds are way too many as we can see in the plot below.
In this notebook we will use a value of 0.01 for contamination which represents the proportion of outliers in the data set. 

<img src="https://raw.githubusercontent.com/clarify/data-science-tutorials/main/media/email_notifications/anomalies.png"/>
Anomaly points when setting contamination to 'auto'. Use contamination = 0.01 to get better results.





In [ ]:
# Set models parameters
print("Outliers fraction is set to: ", outliers_fraction)
rng = np.random.RandomState(100)
model = IsolationForest(n_estimators = 90, max_samples=200, contamination=outliers_fraction, random_state=rng)
model.fit(X)
pred = model.predict(X) 

In [ ]:
# Fast hack to see the score regions. 

X_ = np.column_stack((np.array(df["x"]), np.arange(0, len(df["x"]))))
rng = np.random.RandomState(100)
Xmodel = IsolationForest(n_estimators = 90, max_samples=200, contamination=outliers_fraction, random_state=rng)
Xmodel.fit(X_)

rcParams['figure.figsize'] = 30, 13
l = len(df["x"])
xx, yy = np.meshgrid(np.linspace(0, l, 50), np.linspace(0, l, 50))
Z = Xmodel.decision_function(np.c_[yy.ravel(), xx.ravel()]) # decision_function returns the anomaly scores
Z = Z.reshape(xx.shape)

plt.title("IsolationForest")
plt.contourf(xx, yy, Z, cmap=plt.cm.Blues_r)

b1 = plt.scatter(X_[:, 1], X_[:, 0], c="yellow", s=40, edgecolor="k")
plt.axis("tight")
plt.xlim((0, l))
plt.ylim((0, l))
plt.legend([b1],["training observations"],loc="upper left")
plt.show()

In the plot above we can see an example of different colors of regions. 

The darker the color is, the more likely is that the points in this area are anomaly points. 

These regions where created by using the anomaly scores for x = (0,1, ... 50) and y = (0,1,...50) on the 2d fitted model. 

This way we can have the scores for the hole grid.

Note that we will not use the 2d model since we have only one time series. This plot was created only as an example - as the results of the 1d and 2d model are fairly similar.

In [ ]:
scores = model.decision_function(X)
l = len(scores)

fig = go.Figure(data=go.Scatter(x = np.arange(0,l), y = scores))
fig.update_layout(title = "Score values")
fig.show()

We can also plot the score values from the training set X. Negative scores represent outliers, positive scores represent inliers. If we zoom in the plot, we can easily find which data points are found as anomalies. Note that many lines are very close to y = 0. Therefore the algorithm is not sure if these points are anomalies or not.

In [ ]:
df['anomaly_score'] = pd.Series(pred)
anomaly = df.loc[df['anomaly_score'] == -1, ['date', 'x']] 

print("anomalies found:")
anomaly

In [ ]:
fig = go.Figure(data=go.Scatter(x = np.arange(0,len(pred)), y = pred))
fig.update_layout(title = "predict values")
fig.show()

The `predict` method returns the values 1 or -1. With -1 are marked the data points which corresponde to a negavive score value and with 1 are marked the data points which corresponde to a positive score value.

In the above plot we can see that the same data points which have a negative score value, have a -1 predict value.

In [ ]:
fig = go.Figure(data=go.Scatter(x = df['date'], y = df['x'], mode='markers', name = "Normal values", marker=dict(color='blue', size=4)))
fig.add_traces(go.Scatter(x = anomaly['date'], y = anomaly['x'], textposition='top left',
                          textfont=dict(color='#233a77'),
                          mode='markers+text',
                          name = "Anomaly value",
                          marker=dict(color='red', size=6)))

fig.show()

Last but not least, we plot the 'normal' data points with blue, and the anomaly data points with red.

And save the model so that we can use it on new (streaming) data.

In [ ]:
file = '../data/model.sav'
f = open(file,'x')
f.close()

with open(file, 'wb') as f:
    print("Save model in model file...")
    pickle.dump(model, f)
print("Done!")

As an end note, some of you might be wondering why we didn't mentioned anything about normalizing our data. 
This is a good observation. The first question that we should ask is if it is needed. Since we are using an anomaly detection algorithm, normalizing our data would maybe make it harder to the algorithm to find anomaly points. By scaling our data, it will probably not play a big role, since the way the Isolation Forest algorithm works is by randomly selecting a value between the min and max value and spit them. What the min and max values are doesn't matter, but the bigger the range is, the easier it probably is to find anomaly points. 